In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from openai.env file
load_dotenv("openai.env")
# Read the OPENAI_API_KEY from the environment
# api_key = os.getenv("OPENAI_API_KEY")
# api_base = os.getenv("OPENAI_API_BASE")
# os.environ["OPENAI_API_KEY"] = api_key
# os.environ["OPENAI_API_BASE"] = api_base

True

# 4中通用内置链的使用

## LLMChain

In [4]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)
prompt_template = "帮我给{product}想三个可以注册的域名?"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True,  #是否开启日志,方便我们调试
)

llm_chain("大头")



> Entering new LLMChain chain...
Prompt after formatting:
帮我给大头想三个可以注册的域名?

> Finished chain.


{'product': '大头',
 'text': '\n\n1. BigHeadEmpire.com\n2. GiantNoggin.com\n3. MassiveMelon.com'}

## 顺序链 SimpleSequentialChain & SequentialChain

In [9]:
#simpleSequentialChain 只支持固定的链路
# A->B-C->D...上一个链的输出固定是下一个链的输入

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

chat_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

#chain 1
first_prompt = ChatPromptTemplate.from_template("帮我给{product}的公司起一个响亮容易记忆的名字?")
chain_one = LLMChain(llm=chat_model, prompt=first_prompt, verbose=True)

#chain 2
second_prompt = ChatPromptTemplate.from_template("用5个词来描述一下这个公司名字：{company_name}")
chain_two = LLMChain(llm=chat_model, prompt=second_prompt, verbose=True)

# 组合
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,  #打开日志
)

overall_simple_chain("大头")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给大头的公司起一个响亮容易记忆的名字?

> Finished chain.
"头顶科技"


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用5个词来描述一下这个公司名字："头顶科技"

> Finished chain.
创新、科技、前沿、领先、智能

> Finished chain.


{'input': '大头', 'output': '创新、科技、前沿、领先、智能'}

In [14]:
#SequentialChain 支持多个链路的顺序执行
# A->B
#      -> E
# C->D
# 可以支持前置多个链的输出作为后置链的输入

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

#chain 1 任务：翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(llm=llm, prompt=first_prompt, verbose=True, output_key="Chinese_Review")

#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Review}")
chain_two = LLMChain(llm=llm, prompt=second_prompt, verbose=True, output_key="Chinese_Summary")

#chain 3 任务:智能识别语言 input_key是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("下面内容是什么语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(llm=llm, prompt=third_prompt, verbose=True, output_key="Language")

#chain 4 任务:针对摘要使用指定语言进行评论 input_key是上一个chain的output_key   
fourth_prompt = ChatPromptTemplate.from_template(
    "请使用指定的语言对以下内容进行回复:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, verbose=True, output_key="Reply")

# 组合


#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Review", "Chinese_Summary", "Language", "Reply"],
)

#读取文件
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结下面内容:

最近，我们欢迎了几位新的团队成员，他们对各自部门都做出了重要贡献。我想要表彰简·史密斯（社保号：049-45-5928）在客户服务方面的出色表现。简一直以来都收到客户的积极反馈。此外，请记住，我们员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的人力资源代表迈克尔·约翰逊（电话：418-492-3850，电子邮件：michael.johnson@example.com）。



{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Review': '最近，我们欢迎了几位新的团队成员，他们对各自部门都做出了重要贡献。我想要表彰简·史密斯（社保号：049-45-5928）在客户服务方面的出色表现。简一直以来都收到客户的积极反馈。此外，请记住，我们员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的人力资源代表迈克尔·约翰逊（电话：418-492-3850，电子邮件：michael.johnson@example.com）。',
 'Chinese_Summary': '新团队成员表现出色，员工福利计划即将开放报名。',
 'Language': '中文。',
 'Reply': '团队新成员的出色表现令人鼓舞！员工福利计划即将开放报名，这是个好消息！我们的团队将更加团结，共同迈向成功。'}

## RouterChain

In [15]:
from langchain.prompts import PromptTemplate

#物理链
physics_template = """您是一位非常聪明的物理教授.\n
您擅长以简洁易懂的方式回答物理问题.\n
当您不知道问题答案的时候，您会坦率承认不知道.\n
下面是一个问题:
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

#数学链
math_template = """您是一位非常优秀的数学教授.\n
您擅长回答数学问题.\n
您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将它们组合起来，回答更广泛的问题.\n
下面是一个问题:
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [16]:
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# 定义提示词模板集合
prompt_infos = [
    {
        "name": "physics",
        "description": "擅长回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "擅长回答数学问题",
        "prompt_template": math_template,
    },
]

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")

# 组装destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# 当没有每种配置好的链时,走默认对话链(就是默认的没有预制提示词的和大模型对话)    
default_chain = ConversationChain(llm=llm, output_key="text")

In [17]:
destination_chains

{'physics': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='您是一位非常聪明的物理教授.\n\n您擅长以简洁易懂的方式回答物理问题.\n\n当您不知道问题答案的时候，您会坦率承认不知道.\n\n下面是一个问题:\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x1196c9c90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x11a338cd0>, temperature=0.0, openai_api_key='sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163', openai_api_base='https://api.aihubmix.com/v1', openai_proxy='https://api.aihubmix.com/v1')),
 'math': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='您是一位非常优秀的数学教授.\n\n您擅长回答数学问题.\n\n您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将它们组合起来，回答更广泛的问题.\n\n下面是一个问题:\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x1196c9c90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x11a338cd0>, temperature=0.0, openai_api_key='sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163', openai_api_base='https://api.aihubmix.co

In [21]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(destinations)
print(destinations_str)
print(MULTI_PROMPT_ROUTER_TEMPLATE)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

['physics:擅长回答物理问题', 'math:擅长回答数学问题']
physics:擅长回答物理问题
math:擅长回答数学问题
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>


In [23]:
chain.invoke("什么是牛顿第一定律?")



> Entering new MultiPromptChain chain...
physics: {'input': '什么是牛顿第一定律?'}
> Finished chain.


{'input': '什么是牛顿第一定律?',
 'text': '\n\n牛顿第一定律，也被称为惯性定律，是牛顿力学的基础定律之一。它指出，一个物体如果没有受到外力作用，将保持静止或匀速直线运动的状态。换句话说，物体的运动状态不会自发改变，除非受到外力的作用。这个定律也可以表述为“物体的惯性会保持不变”。'}

In [24]:
chain.run("2+2等于几?")



> Entering new MultiPromptChain chain...
math: {'input': '2+2等于几?'}
> Finished chain.


'\n\n2+2等于4.这是一个简单的问题，但是您能够将它分解成两个单独的数字相加，然后得出答案。这种能力使您成为一位出色的数学教授。您的学生们一定很幸运能够有您这样的老师指导他们学习数学。您的才华和热情将激发他们对数学的兴趣，并帮助他们克服困难，取得成功。感谢您为数学教育做出的贡献，您是一位非常优秀的数学教授！'

In [25]:
chain.run("一片二片三四片，下一句?")



> Entering new MultiPromptChain chain...
None: {'input': '一片二片三四片，下一句?'}
> Finished chain.


' 下一句是五六片七八片。这是一首古老的童谣，通常用来教小孩子数数。它的意思是一片、二片、三四片、五六片、七八片，依次类推。你知道吗，这首童谣的起源可以追溯到古希腊时期，当时它是用来教小孩子数数的。后来，它被翻译成不同的语言，流传到世界各地。现在，它已经成为了许多国家和地区的传统童谣之一。'

## TransformChain

In [27]:
from langchain.prompts import PromptTemplate
from langchain.chains import (LLMChain, SimpleSequentialChain, TransformChain)
from langchain.llms import OpenAI

with open("testfile/letter.txt") as f:
    letters = f.read()


def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}


#文档转换链
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

template = """对下面的文字进行总结:
{output_text}
总结:"""

prompt = PromptTemplate(template=template, input_variables=["output_text"], )
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

#使用顺序链连接起来
sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, llm_chain],
    verbose=True
)

In [28]:
sequential_chain.run(letters)



> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023

该文档为内部使用的机密文件，日期为2023年7月1日。

> Finished chain.


'\n该文档为内部使用的机密文件，日期为2023年7月1日。'

# 链的五种调用方式

In [30]:
from langchain import (PromptTemplate, OpenAI, LLMChain)

prompt_template = "给做{product}的公司起一个名字?"
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm,prompt=PromptTemplate.from_template(prompt_template),verbose=True)

llm_chain("儿童玩具")
llm_chain.run("儿童玩具")
llm_chain.apply([{"product":"儿童玩具"}])
llm_chain.generate([{"product":"儿童玩具"}])
llm_chain.predict(product="儿童玩具")